In [39]:
import requests
import sys

# Querying a Web API with Python and `requests`

## Installing Requests
Install the requests package using the name `requests`:

```python
python -m pip install requests # windows
python3 -m pip install requests # all other OSes
```

In [5]:
# Base URL is a template string which 
# is helpful for adding categories to the URL
URL = "https://v2.jokeapi.dev/joke/{}"

# Available URL Query Parameters

# desired response format
format = ('json', 'xml', 'yaml', 'txt')

# filter out jokes flagged as...
blacklistFlags = (
    "nsfw", "religious", "political", "racist",
    "sexist", "explicit"
)

# Lang code: Joke language (these are available)
lang = ("cs", "de", "en", "es", "fr", "pt")

# Joke type
type = ("single", "twopart")

# contains: case sensitive keyword (search term)

# idRange=number[-number]: limited by the range of jokes

# amount: how many jokes should be included in the response

# categories: note that this must be added to the URL
# If more than one category is desired, separate them
# with commas (e.g. ','.join(categories))
categories = (
    "Any", "Misc", "Programming", "Dark"
    "Pun", "Spooky", "Christmas",
)

## Example Usage
For this example, we will request a single joke with the following attributes:
- category: Pun
- lang: English (en)
- type: single
- blacklistFlags: all flags

### Prepare Query
- The requests library will convert a dictionary to the appropriate URL-encoded query string *and* append the query to the URL.
- The API endpoint URL must be modified explicitly to add one or more categories.

In [19]:
category = "Pun"
query = {
    "format": "json",
    "blacklistFlags": ','.join(blacklistFlags),
    "lang": "en",
    "type": "single",
    "amount": 1,
}

url = URL.format(category)

### Make the Request
- The Joke API endpoint we are using accepts `GET` requests. Use `requests.get()`
- `requests.get()` accepts a dictionary passed as the keyword argument `params`. 

In [20]:
response = requests.get(url, params=query)

### Check the Response Status
The response will contain a *status code*.  The Joke API responds with one of the following. Ideally, this will be `200`, but be prepared for other response codes. Anything other than `200` means that something went wrong.  The response will *not* contain a joke in this case.

- 200: OK
- 400: Bad Request
- 403: Forbidden
- 404: Not Found
- 413: Payload too large (must be less than 5120 Bytes)
- 414: URI too long (URI/URL must be <250 characters)
- 429: Too Many requests (limited to 120 / minute)
- 500: Internal Server Error
- 529: Origin Unreachagble (server offline)

Use `response.raise_for_status()` to raise errors in the 4xx-5xx range. Since this will raise an error if it occurred, there really is no reason to check the status afterward.

In [21]:
# Raise an error if it occurred
response.raise_for_status()

# inspect the response
print(response.text)

{
    "error": false,
    "category": "Pun",
    "type": "single",
    "joke": "Today, my son asked \"Can I have a book mark?\" and I burst into tears.\n11 years old and he still doesn't know my name is Brian.",
    "flags": {
        "nsfw": false,
        "religious": false,
        "political": false,
        "racist": false,
        "sexist": false,
        "explicit": false
    },
    "id": 200,
    "safe": true,
    "lang": "en"
}


### Inspecting the Response Text
The response text is a JSON string.  Typically, in a program, we prefer to convert this to a dictionary. 

### Convert the Response to `dict`
Use the `.json()` method of the response object to get a dictionary with the response.

In [24]:
joke_dict = response.json()

In [25]:
print(joke_dict['joke'])

Today, my son asked "Can I have a book mark?" and I burst into tears.
11 years old and he still doesn't know my name is Brian.


### Get Another Joke


In [26]:
response = requests.get(url, params=query)
response.raise_for_status()
json_dict = response.json()
json_dict

{'error': False,
 'category': 'Pun',
 'type': 'single',
 'joke': "I'm reading a book about anti-gravity. It's impossible to put down!",
 'flags': {'nsfw': False,
  'religious': False,
  'political': False,
  'racist': False,
  'sexist': False,
  'explicit': False},
 'id': 126,
 'safe': True,
 'lang': 'en'}

## Some Errors Aren't HTTP Errors
In the event that the filter parameters do not yield a search result, the API will note this in the response JSON--not an HTTP status code.  

To handle such cases, we need to check the value of teh `error` key returned in the response.  

The following example demonstrates this.

In [27]:
query['contains'] = 'chicken'
response = requests.get(url, params=query)
response.raise_for_status()

joke_dict = response.json()

if joke_dict['error']:
    print(
        "A Joke Filter Error occurred. See JSON response",
        file=sys.stderr
    )
    print(joke_dict)
else:
    print(joke_dict['joke'])
    


{'error': True, 'internalError': False, 'code': 106, 'message': 'No matching joke found', 'causedBy': ['No jokes were found that match your provided filter(s).'], 'additionalInfo': 'Error while finalizing joke filtering: No jokes were found that match your provided filter(s).', 'timestamp': 1656678824195}


A Joke Filter Error occurred. See JSON response


## Two-Part Jokes
Two-part jokes return the joke in a different way.  The following example demonstrates this

In [28]:
query = {
    "format": "json",
    "blacklistFlags": ','.join(blacklistFlags),
    "lang": "en",
    "type": "twopart",
    "amount": 1,
}

In [29]:
response = requests.get(url, params=query)
response.raise_for_status()

json_dict = response.json()
json_dict

{'error': False,
 'category': 'Pun',
 'type': 'twopart',
 'setup': 'Why did the banana go see a doctor?',
 'delivery': "Because it wasn't peeling well.",
 'flags': {'nsfw': False,
  'religious': False,
  'political': False,
  'racist': False,
  'sexist': False,
  'explicit': False},
 'id': 256,
 'safe': True,
 'lang': 'en'}

## Getting the Two-Parts
As you can see, the joke is split into two parts:
- setup
- delivery

In [30]:
print(json_dict['setup'])
print(json_dict['delivery'])

Why did the banana go see a doctor?
Because it wasn't peeling well.


## Handle Either Type
If the `type` parameter is not specified, we need to check the joke type in the response. 

In [33]:
def print_joke(joke):
    """Prints the joke contained in the joke argument. 
    Handles single and two-part jokes
    """
    if joke['type'] == 'twopart':
        joke_str = joke['setup'] + '\n' + joke['delivery']
    else:
        joke_str = joke['joke']
        
    print(joke_str)

In [34]:
query = {
    "format": "json",
    "blacklistFlags": ','.join(blacklistFlags),
    "lang": "en",
    "amount": 1,
}

response = requests.get(url, params=query)
response.raise_for_status()

joke_dict = response.json()

if not joke_dict['error']:
    print_joke(joke_dict)

I'm reading a book about anti-gravity. It's impossible to put down!


## Print Random Joke
The Joke API returns a random joke. If we set the category to 'Any', we essentially get a random joke.  Here, the flags are all left in place.

Moreover, we bundle all of this into a function.

In [35]:
def print_random_joke():
    """Requests a joke from Joke API and prints it"""

    url = "https://v2.jokeapi.dev/joke/Any"
    query = {
        "format": "json",
        "blacklistFlags": ','.join(blacklistFlags),
        "lang": "en",
        "amount": 1,
    }
    response = requests.get(url, params=query)
    response.raise_for_status()
    
    joke = response.json()
    
    if joke['error']:
        raise ValueError(f'Filter error occured:\n{joke}')
    
    print_joke(joke)
    

In [36]:
print_random_joke()

A programmer puts two glasses on his bedside table before going to sleep.
A full one, in case he gets thirsty, and an empty one, in case he doesn't.


In [37]:
print_random_joke()

I'm reading a book about anti-gravity. It's impossible to put down!


In [38]:
print_random_joke()

Why did the programmer quit his job?
Because he didn't get arrays.
